##### Copyright 2018 The TensorFlow Authors.


In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텍스트 로드

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운로드</a>
  </td>
</table>

이 튜토리얼에서는 `tf.data.TextLineDataset` 을 사용하여 텍스트 파일에서 예제를 로드하는 방법에 대한 예제를 제공합니다. `TextLineDataset` 은 텍스트 파일에서 데이터 집합을 만들도록 설계되었습니다, 각 예제는 원본 파일의 텍스트 체계입니다. 이것은 주로 라인 기반의 텍스트 데이터 (예를 들어, 시 또는 오류 로그)에 유용합니다.

이 튜토리얼에서는 동일한 작업에 대한 세 가지 다른 영어 번역 인 Homer 's Illiad를 사용하고 한 줄의 텍스트로 번역기를 식별하는 모델을 학습합니다.

##설치 

In [0]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

세 가지 번역의 본문은:

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

이 튜토리얼에서 사용 된 텍스트 파일은 대부분의 문서 헤더 및 바닥 글, 줄 번호, 챕터 제목 등 을 제거하는 일반적인 전처리 작업을 거쳤습니다. 약간 변경된 이 파일들을 부분적으로 다운받아봅시다.

In [0]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

## 데이터셋에 텍스트 로드

파일을 반복하면서 각 파일을 자체 데이터 세트에 로드합니다.

각 예제는 개별적으로 레이블을 지정해야 하므로 `tf.data.Dataset.map` 을 사용하여 각 라벨러기능을 적용합니다. 이런식으로 데이터셋의 모든 예제를 반복하여 (`예시, 레이블`) 쌍을 반환합니다.

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

레이블로 지정된 데이터셋을 단일 데이터셋으로 결합하고 섞습니다.


In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [0]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

`tf.data.Dataset.take`를 사용하여 `(예시, 레이블)` 쌍의 모습을 확인하여 출력 할 수 있습니다. `numpy` 속성은 각 텐서의 값을 보여줍니다.

In [0]:
for ex in all_labeled_data.take(5):
  print(ex)

## 텍스트 줄을 숫자로 인코딩

머신 러닝 모델은 단어가 아닌 숫자에 대해 작동하므로 문자열 값을 숫자 목록으로 변환해야 합니다. 그렇게 하려면, 각각의 고유한 단어를 고유한 정수로 매핑해봅시다.

### 언어 구축하기

먼저, 텍스트를 개별 고유 단어 모음으로 토큰 화하여 언어를 구축합시다.. TensorFlow와 Python에서 이를 수행하는 몇 가지 방법이 있습니다. 이 튜토리얼 에서는 :

1. 각 예제의 `numpy` 값을 반복합니다.
2.  `tfds.features.text.Tokenizer` 를 사용하여 토큰으로 분할합니다.
3. 이 토큰들을 Python셋으로 모아서, 중복 하는 것을 제거합니다.
4. 나중에 사용하기 위해서 어휘의 크기를 측정합니다.

In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

### 인코딩 예제

 `vocabulary_set` 을 `tfds.features.text.TokenTextEncoder`에 전달하여  인코더를 작성합시다.  인코더의 `encode` 메소드는 텍스트 문자열을 받아서 정수열을 반환합니다.

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

한 줄로 실행하여 출력이 어떻게 나오는지 확인할 수 있습니다.

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

In [0]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

이제 `tf.py_function` 으로 행갈이를 해서 데이터셋의 `맵` 메소드에 전달하여 데이터셋에서 인코더를 실행합니다.

In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

 `Dataset.map` 을 사용하여 이 함수를 데이터셋의 각 요소에 적용하려고 합니다. `Dataset.map` 은 그래프 모드에서 실행됩니다.

* 그래프 텐서는 값을 가지지 않습니다. 
* 그래프 모드에서만 TensorFlow 작업과 함수를 사용할 수 있습니다.  

따라서 이 함수를 바로 `.map` 할 수 없습니다. `tf.py_function`으로 행갈이를 해야합니다. `tf.py_function`은 값과 `.numpy()` 메소드를 사용하여 액세스하는 일반 텐서를 행갈이된 파이썬 함수에 전달합니다.

In [0]:
def encode_map_fn(text, label):
  # py_function은 반환된 tensor들의 모양을 설정해주지 않습니다.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets`은 모든 구성요소가 모양세트를 가지고 있을 경우 가장 잘 작동합니다.
  #  따라서 모양을 설정해줘야 합니다.: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## 데이터셋을 테스트 및 트레이닝 배치로 분할

`tf.data.Dataset.take` 및 `tf.data.Dataset.skip`을 사용하여 작은 테스트 데이터셋과 더 큰 트레이닝셋을 작성합시다..

모델로 전달되기 전에 데이터셋을 일괄 처리해야합니다. 일반적으로 배치 내부의 예제들은 크기와 모양이 같아야 합니다. 그러나 이 데이터셋의 예제들은 모두 같은 크기가 아닙니다. 각 텍스트 줄의 단어 수는 다릅니다. 따라서 `batch` 대신 `tf.data.Dataset.padded_batch` 를 사용하여 예제들을 동일한 크기로 채웁니다.

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

이제 `test_data` 및 `train_data` 는 (`예시, 레이블`) 쌍의 모음이 아니라 배치 모음입니다. 각 배치는 배열로 표시되는 한 쌍의 (*많은 예시들*, *많은 레이블들*)입니다.

설명:

In [0]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

 새로운 토큰 인코딩 (패딩에 사용되는 0 인코딩)을 도입했기 때문에 언어 크기가 1 씩 증가했습니다.

In [0]:
vocab_size += 1

## 모델 구축


In [0]:
model = tf.keras.Sequential()

첫 번째 레이어는 정수 표현을 고밀도 벡터 임베딩으로 변환합니다. [word embeddings tutorial](../text/word_embeddings.ipynb) 또는 다른 튜토리얼들을 참고합시다. 

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

다음 레이어는  [Long Short-Term Memory](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)레이어로, 모델이 문맥에서 단어를 다른 단어와 이해할 수 있도록 합니다. LSTM의 양방향 래퍼는 이전 및 이후의 데이터 포인트와 관련하여 데이터 포인트를 학습하는 데 도움을 줍니다.

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

마지막으로 우리는 일련의 하나 이상으로 조밀하게 연결된 레이어를 갖게 되며, 그 마지막 레이어는 바로 출력 레이어입니다. 출력 레이어는 모든 라벨에 대한 확률을 생성합니다. 확률이 가장 높은 출력 레이어는 예제 라벨의 모델 예측입니다.

In [0]:
# 하나이상으로 조밀하게 연결된 레이어들.
# for문의 리스트를 수정하여 레이어의 사이즈를 알아봅시다.  
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# 출력 레이어. 첫 번째 인수는 레이블 수 입니다.
model.add(tf.keras.layers.Dense(3))

마지막으로 모델을 컴파일합니다. 소프트맥스 분류 모델의 경우 손실 함수로 `sparse_categorical_crossentropy`를 사용합니다. 다른 옵티마이저를 사용해 볼 수 있지만 `adam` 을 일반적으로 사용합니다.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델 트레이닝

이 데이터에서 실행되는 모델은 적절한 결과 (약 83 %)를 제공합니다..

In [0]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [0]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))